# Global Navigation

The goal of this part is to create a trajectory that will allow the Thymio to pass through several interest point and come back to his initial position.

The trajectory has to ensure that the Thymio avoid the global obstacles detected by the camera.

## Pass planning : Process flow

The creation of our trajectory is composed of the following steps :

- Dilatation of the obstacles to avoid collisions.

- Visibility graph computation by using the previously dilated obstacles.

- Pass planning computation by using the visibility graph and the interest point positions.

### Obstacle dilatation

The first algorithm that we used to dilate the obstacle was composed of the following steps :

- Compute the centroid of each obstacles.

- For each obstacle, map the original contour points to new coordinates where (0, 0) is the centroid.

- Scale the position in this new reference frame.

- Map it back to previous coordinates by adding back position of centroids.

To compute the centroid of our obstacles, we use the moment functionality provided in the OpenCV library. In particular, we will use the following relations :

$$M_{ij} = \sum_{x}\sum_{y} x^iy^jI(x, y)$$ 

$$\bar x = \frac{M_{10}}{M_{00}}$$

$$\bar y = \frac{M_{01}}{M_{00}}$$


For more details, see [Image moments](https://en.wikipedia.org/wiki/Image_moment).


In [1]:
import cv2

In [1]:
def computeCentroid(contours):
    """Given the contours of a set of polygons, compute their respective centroids

    Parameters
    ----------
    contours : list of list of list
        The camera detect several obstacles/interest points
        Each has several extremities
        Each extremity has (x, y) coordinates

    Returns
    -------
    centroids : list of list
        Each polygon has a centroid composed of (x, y) coordinates
    """

    centroids = []
    for obstacle in contours:
        M = cv2.moments(obstacle)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        centroids.append([cx, cy])

    return centroids

Knowing how to compute centroids, we can now follow our previous algorithm to dilate obstacles :

In [3]:
# Define
X = 0
Y = 1

In [5]:
def dilateObstacles(contours, scalingFactor):
    """Given the contours of obstacles and the scaling factor, dilate these contours 

    Parameters
    ----------
    contours : list of list of list
        The camera detect several obstacles
        Each obstacle has several extremities
        Each extremity has (x, y) coordinates

    scalingFactor : float
        Define how much the obstacle's contours should be dilated
        scalingFactor = 1.4 -> 40 % of dilatation

    Returns
    -------
    contoursMapped : list of list of list
        Same structure as contours, each extremity's coordinate has been dilated
    """

    # Compute centroid of each obstacle
    centroids = computeCentroid(contours)

    
    # For each obstacle, map the original contour points to new coordinates where (0, 0) is the centroid
    contoursMapped = [[] for _ in range(len(contours))]

    for obstacle, i in zip(contours, range(len(contours))):
        for extremity in obstacle:
            contoursMapped[i].append([extremity[0][X] - centroids[i][X], extremity[0][Y] - centroids[i][Y]])


    # Scale position
    for obstacle, i in zip(contoursMapped, range(len(contoursMapped))):
        for j in range(len(obstacle)):
            contoursMapped[i][j][X] *= scalingFactor
            contoursMapped[i][j][Y] *= scalingFactor


    # Map it back to previous coordinates by adding back position of centroids
    for obstacle, i in zip(contoursMapped, range(len(contoursMapped))):
        for j in range(len(obstacle)):
            contoursMapped[i][j][X] = int(contoursMapped[i][j][X] + centroids[i][X])
            contoursMapped[i][j][Y] = int(contoursMapped[i][j][Y] + centroids[i][Y])
    
    return contoursMapped 

To see the result of our algorithm, we will use the following function (note that this function will be also used to plot other results computed in the global Navigation part) :

In [15]:
import matplotlib.pyplot as plt

In [16]:
def printGlobalNavigation(contours, contoursMapped, possibleDisplacement = {}, interestPoints = [], trajectory = []):
    """Plot the original contours and the dilated contours using matplotlib
       Plot the visibility graph if possibleDisplacement is given 
       Plot the Thymio's point of interest if interestPoints is given
       Plot the Thymio's path if the trajectory is given

    Parameters
    ----------
    contours : list of list of list
        The camera detect several obstacles
        Each obstacle has several extremities
        Each extremity has (x, y) coordinates

    contoursMapped : list of list of list
        Same structure as contours, each extremity's coordinate has been dilated

    possibleDispacement : dictionary
        Each extremity point has several visible points, i.e possible destinations for the Thymio

    interestPoints : list of list
        Each point of interest has (x, y) coordinates, i.e locations where the thymio need to go

    trajectory : list of list
        Each point of the trajectory has (x, y) coordinates

    Returns
    -------

    """

    xOriginal = []
    yOriginal = []
    xDilated = []
    yDilated = []

    for obstacleOriginal in contours:
        for extremityOriginal in obstacleOriginal:
            xOriginal.append(extremityOriginal[X])
            yOriginal.append(extremityOriginal[Y])

        xOriginal.append(obstacleOriginal[0][X])
        yOriginal.append(obstacleOriginal[0][Y])

        plt.plot(xOriginal, yOriginal, 'b')

        xOriginal.clear()
        yOriginal.clear()

    
    for obstacleDilated in contoursMapped:
        for extremityDilated in obstacleDilated:
            xDilated.append(extremityDilated[X])
            yDilated.append(extremityDilated[Y])

        xDilated.append(obstacleDilated[0][X])
        yDilated.append(obstacleDilated[0][Y])

        plt.plot(xDilated, yDilated, 'm')

        xDilated.clear()
        yDilated.clear()


    if possibleDisplacement:
        for extremity in possibleDisplacement:
            for visiblePoint in possibleDisplacement[extremity]:
                plt.plot([extremity[X], visiblePoint[X]], [extremity[Y], visiblePoint[Y]], 'm')

    
    if interestPoints:
        for point in interestPoints:
            plt.plot([point[X]], [point[Y]], 'kx', markersize=12)


    if trajectory:
        for i in range (1, len(trajectory)):
            plt.arrow(trajectory[i-1][X], trajectory[i-1][Y], trajectory[i][X] - trajectory[i-1][X], trajectory[i][Y] - trajectory[i-1][Y], head_width=8, length_includes_head=True, color  = 'k', width = 2)

    plt.show()

In [10]:
import numpy as np

# Vision's input : extremities of each obstacles
contours = [np.array([[[504, 236]], [[495, 199]], [[380, 212]], [[438, 274]]], dtype=np.int32), 
            np.array([[[170, 195]], [[254, 275]], [[296, 238]], [[235, 194]]], dtype=np.int32), 
            np.array([[[302, 168]], [[290, 182]], [[294, 199]], [[312, 209]], [[333, 203]], [[337, 175]]], dtype=np.int32), 
            np.array([[[228, 151]], [[301, 102]], [[219,  89]]], dtype=np.int32), 
            np.array([[[481, 130]], [[457,  66]], [[360,  81]], [[434, 150]]], dtype=np.int32)]

# Dilate obstacles and print them
contoursMapped = dilateObstacles(contours, scalingFactor = 1.4)
printGlobalNavigation(contours, contoursMapped)

IndexError: index 1 is out of bounds for axis 0 with size 1